# Imports

In [34]:
import sys
import os
from google.colab import drive
from google.colab import files
from dotenv import load_dotenv
import json
import warnings
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
%matplotlib inline

# Bootstrap

In [35]:
np.random.seed(31071967)

# Find and load the .env file from the current or parent directories
load_dotenv()

drive.mount('/content/drive')

with open(f"{os.getenv('PROJECT_PATH')}/src/config.json", 'r') as f:
    project_config = json.load(f)
    project_config.pop('_comment', None)
    project_config.pop('_note', None)
    project_config.pop
    f.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
project_config['STOCKS'] = project_config['STOCKS'].split()
project_config['TKL'] = project_config['STOCKS'][0]

In [37]:
if project_config['chain_notebooks'] == '1':

  !pip install papermill
  !pip install nbconvert
  !pip install nbformat
  !pip install IPython

  import papermill as pm
  import nbformat
  from nbconvert import HTMLExporter
  from IPython.display import HTML, display

  input_file = f"{os.getenv('PROJECT_PATH')}{project_config['notebooks_directory']}{project_config['notebook1']}"
  output_file = f"{os.getenv('PROJECT_PATH')}{project_config['output_directory']}{project_config['output1']}"

  # --- Execute the proviuse notebook with parameters ---
  pm.execute_notebook(
      input_path = input_file,
      output_path = output_file,
      log_output=False,  # don't print logs while running
      progress_bar=True
  )

  # --- Convert the executed notebook to HTML ---
  nb = nbformat.read(output_file, as_version=4)
  html_exporter = HTMLExporter()
  html_exporter.template_name = "lab"  # modern look; alternatives: 'classic', 'basic'
  body, _ = html_exporter.from_notebook_node(nb)

  # --- Display the HTML result inline ---
  display(HTML(body))

In [38]:
tickers_yf = {
    "y"     : f"{project_config['TKL']}",
    "NASDAQ"       : "^IXIC",
    "SP500"        : "^GSPC",
    "Gold"         : "GC=F",
    "Oil"          : "CL=F",
    "RealEstate"   : "VNQ",
    "InflationExp": "^TNX"
}

desired_order = [
    "Date",
    "y",
    "NASDAQ",
    "SP500",
    "Oil",
    "Gold",
    "RealEstate",
    "InflationExp",
]

In [39]:
import yfinance as yf
import pandas as pd
from pandas_datareader import data as pdr

from datetime import date, timedelta
end_date = date.today() - timedelta(days=1)
start_date = end_date - timedelta(days=int(project_config["HISTORY_DEPTH"]))

if project_config['TKL'] == 'TNYA':
  start_date = pd.to_datetime("31.07.2022", format="%d.%m.%Y")

# ---- DOWNLOAD FROM YAHOO FINANCE ----
ts_yf = yf.download(
    tickers=list(tickers_yf.values()),
    start=start_date,
    end=end_date,
    auto_adjust=True
)["Close"]

# rename columns to readable names
rename_map = {v: k for k, v in tickers_yf.items()}
ts_yf = ts_yf.rename(columns=rename_map)

# Fill missing daily values for macro data (monthly)
ts_yf = ts_yf.fillna(method='ffill').fillna(method='bfill')
ts_yf = ts_yf.reset_index().rename(columns={"Date": "Date",})

print(f"\n\nDataset for y={project_config['TKL']}")
display(ts_yf.head(1))
display(ts_yf.tail(1))
ts_yf.info()

[*********************100%***********************]  7 of 7 completed



Dataset for y=AAPL


Ticker,Date,y,Oil,Gold,RealEstate,SP500,NASDAQ,InflationExp
0,2006-10-18,2.235234,57.650002,588.900024,33.827171,1365.800049,2337.149902,4.764


Ticker,Date,y,Oil,Gold,RealEstate,SP500,NASDAQ,InflationExp
4823,2025-12-16,274.609985,55.27,4304.5,89.07,6800.259766,23111.460938,4.149


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4824 entries, 0 to 4823
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          4824 non-null   datetime64[ns]
 1   y             4824 non-null   float64       
 2   Oil           4824 non-null   float64       
 3   Gold          4824 non-null   float64       
 4   RealEstate    4824 non-null   float64       
 5   SP500         4824 non-null   float64       
 6   NASDAQ        4824 non-null   float64       
 7   InflationExp  4824 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 301.6 KB


In [40]:
df = ts_yf.copy()
df_orig = ts_yf.copy()

# Keep Date
date_col = df["Date"]

# Targets
y_col = ['y']

# Features
X_cols = df.drop(columns=y_col+['Date']).columns

# Initialize scalers
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# Scale
df_X_scaled = pd.DataFrame(X_scaler.fit_transform(df[X_cols]),
                           columns=X_cols, index=df.index)

df_y_scaled = pd.DataFrame(y_scaler.fit_transform(df[y_col]),
                           columns=y_col, index=df.index)

# Rebuild dataframe
df = pd.concat([date_col, df_X_scaled, df_y_scaled], axis=1)

df = df[desired_order]
df_orig = df_orig[desired_order]

del df_X_scaled, df_y_scaled

display(df.tail(1))
display(df_orig.tail(1))

,Date,y,NASDAQ,SP500,Oil,Gold,RealEstate,InflationExp
4823,2025-12-16,0.959219,0.96267,0.983815,0.507872,0.991509,0.877166,0.768583


Ticker,Date,y,NASDAQ,SP500,Oil,Gold,RealEstate,InflationExp
4823,2025-12-16,274.609985,23111.460938,6800.259766,55.27,4304.5,89.07,4.149


In [41]:
def generate_ts_features(df, lags=[2,3,4,5,6,10,22,66], windows=[5,10,22,66]):

  # Identify columns to generate features for (excluding 'index' column which is the date)
  base_cols = [c for c in df.columns if c not in ['Date','index']]

  for base_col in base_cols:

    for lag in lags:
      df[f"{base_col}_lag_{lag}"] = df[base_col].shift(lag)

  # Generate rolling window statistics for the current base_col
    for window in windows:
      df[f"{base_col}_min_{window}"] = df[base_col].rolling(window=window).min()
      df[f"{base_col}_max_{window}"] = df[base_col].rolling(window=window).max()
      df[f"{base_col}_mean_{window}"] = df[base_col].rolling(window=window).mean()
      df[f"{base_col}_std_{window}"]  = df[base_col].rolling(window=window).std()
      df[f"{base_col}_diff_{window}"] = df[base_col].diff(window)
      df[f"{base_col}_pct_{window}"] = df[base_col].pct_change(window)

  df['y_next'] = df['y'].shift(-1)                            # y_next = tomorrow's y (close price)
  df.loc[df.index[-1], 'y_next'] = df.loc[df.index[-1], 'y']  # the TARGET cell. y_next tomorrow = y today
  df = df.fillna(method='ffill').fillna(method='bfill')

  return df

df = generate_ts_features(df)
df_orig = generate_ts_features(df_orig)

# Replace infinite values with NaN in X_train and y_train
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(method='ffill').fillna(method='bfill')

display(df.tail(1))
display(df_orig.tail(1))

,Date,y,NASDAQ,SP500,Oil,Gold,RealEstate,InflationExp,y_lag_2,y_lag_3,...,InflationExp_std_22,InflationExp_diff_22,InflationExp_pct_22,InflationExp_min_66,InflationExp_max_66,InflationExp_mean_66,InflationExp_std_66,InflationExp_diff_66,InflationExp_pct_66,y_next
4823,2025-12-16,0.959219,0.96267,0.983815,0.507872,0.991509,0.877166,0.768583,0.972143,0.971263,...,0.012551,0.007791,0.010241,0.727311,0.778058,0.756258,0.013259,0.01853,0.024705,0.959219


Ticker,Date,y,NASDAQ,SP500,Oil,Gold,RealEstate,InflationExp,y_lag_2,y_lag_3,...,InflationExp_std_22,InflationExp_diff_22,InflationExp_pct_22,InflationExp_min_66,InflationExp_max_66,InflationExp_mean_66,InflationExp_std_66,InflationExp_diff_66,InflationExp_pct_66,y_next
4823,2025-12-16,274.609985,23111.460938,6800.259766,55.27,4304.5,89.07,4.149,278.279999,278.029999,...,0.059606,0.037,0.008998,3.953,4.194,4.09047,0.062965,0.088,0.02167,274.609985


In [42]:
X = df.drop(columns=['Date','y_next'])
y = df['y_next']

X_train, X_test = X.iloc[:-2], X.iloc[-1:]
y_train, y_test = y.iloc[:-2], y.iloc[-1:]

del X,y

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4822, 231), (1, 231), (4822,), (1,))

In [43]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

xgb = XGBRegressor(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror")

xgb.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
pred = xgb.predict(X_test)
pred_orig = y_scaler.inverse_transform(pred.reshape(-1, 1))[0, 0] # Corrected to use NumPy indexing

df.loc[df.index[-1], 'y_next'] = pred
df_orig.loc[df_orig.index[-1], 'y_next'] = pred_orig

print(f"{project_config['TKL']} {df['Date'].iloc[-1].date()} ${pred_orig:.2f}")
display(df.tail(1))
display(df_orig.tail(1))

In [ ]:
import matplotlib.pyplot as plt
import xgboost # Import the xgboost module

# Plot feature importance based on "weight" (number of times a feature appears in a tree)
plt.figure(figsize=(10, 6))
xgboost.plot_importance(xgb, max_num_features=20) # Pass the xgb regressor object
plt.title("Feature Importance (Weight)")
plt.show()

# 1. Get feature importance by weight
importance_dict = xgb.get_booster().get_score(importance_type='weight')

# 2. Convert to DataFrame
df_importance = pd.DataFrame(list(importance_dict.items()), columns=['Feature', 'Score'])

# 3. Sort high → low
df_importance = df_importance.sort_values(by='Score', ascending=False)

# 4. Compute threshold = 10% of top feature
top_score = df_importance['Score'].iloc[0]
threshold = top_score * 0.05   # 5%

# 5. Select only strong features
df_top = df_importance[df_importance['Score'] >= threshold]

# 6. Convert to list
top_weight_features_list = df_top['Feature'].tolist()[:10:]

print("Weight Threshold:", threshold)
print("Selected Weight Features:", top_weight_features_list)

In [ ]:
# OPTIONAL: Plot based on "gain" (average gain of splits which use the feature)
# "Gain" is often more accurate for finding what actually drives the prediction.
plt.figure(figsize=(10, 6))
xgboost.plot_importance(xgb, importance_type='gain', max_num_features=20) # Pass the xgb regressor object
plt.title("Feature Importance (Gain)")
plt.show()

# 1. Build importance DataFrame
importance_dict = xgb.get_booster().get_score(importance_type='gain')
df_importance = pd.DataFrame(list(importance_dict.items()), columns=['Feature', 'Score'])

# 2. Sort high to low
df_importance = df_importance.sort_values(by='Score', ascending=False)

# 3. Compute threshold = 10% of top feature
top_score = df_importance['Score'].iloc[0]
threshold = top_score * 0.05

# 4. Select features with Score >= threshold
df_top = df_importance[df_importance['Score'] >= threshold]

# 5. Extract feature names
top_gain_features_list = df_top['Feature'].tolist()[:10:]

print("Threshold:", threshold)
print("Selected features:", top_gain_features_list)

In [ ]:
top_features = list(set(top_gain_features_list) | set(top_weight_features_list))
top_features

In [ ]:
final_df = df[['Date','y_next'] + top_features]

for col in top_features + ['y_next']:
  final_df[f"{col}_orig"] = df_orig[col]

display(final_df.tail(1))
final_df.info()

In [ ]:
df_csv_path = f"{os.getenv('PROJECT_PATH')}{project_config['data_directory']}{project_config['TKL']}.df.csv"
final_df[:-1:].to_csv(df_csv_path)

# Load dataset file of current TKL

In [ ]:
print(f"Loading dataset for {project_config['TKL']} .....\n")

DATASET = f"{os.getenv('PROJECT_PATH')}{project_config['data_directory']}{project_config['TKL']}.df.csv"
df_all = pd.read_csv(DATASET, index_col=False)
df_all = df_all.drop(columns=["Unnamed: 0"])

cols_orig = [col for col in df_all.columns if col.endswith('_orig')]
cols_normalized = [col for col in df_all.columns if not col.endswith('_orig')]

df = df_all[cols_normalized]
df_orig = df_all[['Date'] + cols_orig]

print(f"\ndf for training")
print(f"-----------------")
display(df.tail(1))

print(f"\ndf for visualization")
print(f"----------------------")
display(df_orig.tail(1))

# Data prep

In [ ]:
def make_datasets(df, X_cols, y_col):

    LOOK_BACK_DAYS = int(project_config['LOOK_BACK_DAYS'])

    X, y = [], []

    for i in range(LOOK_BACK_DAYS, len(df)):
        X.append(df.loc[i-LOOK_BACK_DAYS:i-1, X_cols].values)
        y.append(df.loc[i, y_col].values[0])

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    dates = pd.to_datetime(df['Date'])
    dates_test = dates[-len(y_test):].values

    return X_train, X_test, y_train, y_test, dates_test

# Train Predict

In [ ]:
def train_predict(X_train, X_test, y_train, y_test, features, model_name, epochs=0, batch_size=0):

  from tensorflow.keras.models import Sequential, Model
  from tensorflow.keras.layers import LSTM, Dense, GRU, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Concatenate

  epochs = int(project_config['TRAIN_EPOCS']) if epochs == 0 else epochs
  batch_size = int(project_config['TRAIN_BATCH_SIZE']) if batch_size == 0 else batch_size

  if model_name == 'Parallel.LSTM.GRU':
    input_layer = Input(shape=(X_train.shape[1],X_train.shape[2]))
    lstm_branch = LSTM(units=50, return_sequences=False)(input_layer)
    gru_branch = GRU(units=50, return_sequences=False)(input_layer)
    merged = Concatenate()([lstm_branch, gru_branch])
    dropout = Dropout(0.1, name='dropout_layer')(merged)
    output_layer = Dense(1, activation='sigmoid')(dropout)
    model = Model(inputs=input_layer, outputs=output_layer)

  if model_name == 'Cascase.CNN.GRU':
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2])),
        MaxPooling1D(pool_size=2),
        GRU(50, return_sequences=False),
        Dense(25, activation='relu'),
        Dense(1)
  ])

  if model_name == 'Cascase.CNN.LSTM':
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2])),
        MaxPooling1D(pool_size=2),
        LSTM(50, return_sequences=False),
        Dense(25, activation='relu'),
        Dense(1)
  ])

  if model_name == 'CNN':
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2])),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.2),
        Dense(1)
  ])

  if model_name == 'LSTM':
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1],X_train.shape[2])),
        LSTM(10),
        Dense(1)
  ])

  if model_name == 'GRU':
    model = Sequential([
      GRU(50, return_sequences=True, input_shape=(X_train.shape[1],X_train.shape[2])),
      GRU(10),
      Dense(1)
  ])

  model.compile(optimizer='adam', loss='mse')

  print("\n=====================================================================\n")
  print(f"Model: {model_name}")
  print(f"Features: {features}")
  model.summary()

  history = model.fit(X_train, y_train,
                      epochs=int(project_config['TRAIN_EPOCS']),
                      batch_size=int(project_config['TRAIN_BATCH_SIZE']),
                      validation_data=(X_test, y_test),
                      verbose=1)

  pred = model.predict(X_test)
  unscaled_prediction = y_scaler.inverse_transform(pred.reshape(-1, 1))
  unscaled_y_test = y_scaler.inverse_transform(y_test.reshape(-1, 1))

  from sklearn.metrics import mean_absolute_error
  mae = mean_absolute_error(unscaled_y_test, unscaled_prediction)

  from sklearn.metrics import r2_score
  r2 = r2_score(unscaled_y_test, unscaled_prediction)

  print(f"Model: {model_name}")
  print(f"Features: {features}")
  print(f"MAE: {mae:.4f}")
  print(f"R²: {r2:.4f}")

  return unscaled_prediction, unscaled_y_test, model

# Racing Models

In [ ]:
X_cols_full = [col for col in df.columns if col not in ['Date','y_next']]
X_cols_exho = [col for col in df.columns if '_' not in col and col != 'Date']
X_cols_tkl  = ['y']
y_col       = ['y_next']

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

def race_models(tkl, models_to_try):

  results_df = pd.DataFrame(columns=[ "model_name", "X_cols",  "prediction", "mae", "r2", "model" ])

  for X_cols in [X_cols_full, X_cols_exho, X_cols_tkl]:

    X_train, X_test, y_train, y_test, dates_test = make_datasets(df, X_cols, y_col)

    if 'Parallel.LSTM.GRU' in models_to_try:
      unscaled_prediction, unscaled_y_test, model = train_predict(X_train, X_test, y_train, y_test, X_cols, model_name="Parallel.LSTM.GRU")
      results_df.loc[len(results_df)] = {
        "model_name": "Parallel.LSTM.GRU",
        "X_cols": X_cols,
        "prediction": unscaled_prediction,
        "mae": mean_absolute_error(unscaled_y_test, unscaled_prediction),
        "r2": r2_score(unscaled_y_test, unscaled_prediction),
        "model": model
        }

    if 'Cascase.CNN.GRU' in models_to_try:
      unscaled_prediction, unscaled_y_test, model = train_predict(X_train, X_test, y_train, y_test, X_cols, model_name="Cascase.CNN.GRU")
      results_df.loc[len(results_df)] = {
        "model_name": "Cascase.CNN.GRU",
        "X_cols": X_cols,
        "prediction": unscaled_prediction,
        "mae": mean_absolute_error(unscaled_y_test, unscaled_prediction),
        "r2": r2_score(unscaled_y_test, unscaled_prediction),
        "model": model
        }

    if 'Cascase.CNN.LSTM' in models_to_try:
      unscaled_prediction, unscaled_y_test, model = train_predict(X_train, X_test, y_train, y_test, X_cols, model_name="Cascase.CNN.LSTM")
      results_df.loc[len(results_df)] = {
        "model_name": "Cascase.CNN.LSTM",
        "X_cols": X_cols,
        "prediction": unscaled_prediction,
        "mae": mean_absolute_error(unscaled_y_test, unscaled_prediction),
        "r2": r2_score(unscaled_y_test, unscaled_prediction),
        "model": model
        }

    if 'CNN' in models_to_try:
      unscaled_prediction, unscaled_y_test, model = train_predict(X_train, X_test, y_train, y_test, X_cols, model_name="CNN")
      results_df.loc[len(results_df)] = {
        "model_name": "CNN",
        "X_cols": X_cols,
        "prediction": unscaled_prediction,
        "mae": mean_absolute_error(unscaled_y_test, unscaled_prediction),
        "r2": r2_score(unscaled_y_test, unscaled_prediction),
        "model": model
        }

    if 'GRU' in models_to_try:
      unscaled_prediction, unscaled_y_test, model = train_predict(X_train, X_test, y_train, y_test, X_cols, model_name="GRU")
      results_df.loc[len(results_df)] = {
        "model_name": "GRU",
        "X_cols": X_cols,
        "prediction": unscaled_prediction,
        "mae": mean_absolute_error(unscaled_y_test, unscaled_prediction),
        "r2": r2_score(unscaled_y_test, unscaled_prediction),
        "model": model
      }

    if 'LSTM' in models_to_try:
      unscaled_prediction, unscaled_y_test, model = train_predict(X_train, X_test, y_train, y_test, X_cols, model_name="LSTM")
      results_df.loc[len(results_df)] = {
        "model_name": "LSTM",
        "X_cols": X_cols,
        "prediction": unscaled_prediction,
        "mae": mean_absolute_error(unscaled_y_test, unscaled_prediction),
        "r2": r2_score(unscaled_y_test, unscaled_prediction),
        "model": model
      }

  return results_df, dates_test, unscaled_y_test

In [ ]:
def plot_results(results_df, dates_test, unscaled_y_test):

  import matplotlib.dates as mdates # Added import

  for model_name in results_df.model_name.unique():
    plt.figure(figsize=(10,6))
    plt.title(f"{project_config['TKL']} {model_name}")
    # Note: unscaled_y_test and dates_test are assumed to be defined in the global scope
    # and represent the test data for comparison with all predictions.
    plt.plot(dates_test, unscaled_y_test, label=f"Actual", linewidth=2, color='black')

    # Corrected iteration: iterate over rows using .iterrows()
    for index, entry in results_df[results_df.model_name == model_name].iterrows():
      plt.plot(dates_test, entry.prediction, label=f"features: {entry.X_cols}", linestyle='--')

    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.legend()
    plt.show()

  # Convert lists in 'X_cols' to tuples for unique identification
  # This is necessary because lists are unhashable, causing TypeError with .unique()
  results_df['X_cols_tuple'] = results_df['X_cols'].apply(tuple)

  for features_tuple in results_df.X_cols_tuple.unique():
    plt.figure(figsize=(10,6))
    # Convert tuple back to list for display purposes in the title
    features = list(features_tuple)
    plt.title(f"{project_config['TKL']} {features}")
    # Note: unscaled_y_test and dates_test are assumed to be defined in the global scope
    # and represent the test data for comparison with all predictions.
    plt.plot(dates_test, unscaled_y_test, label=f"Actual", linewidth=2, color='black')

    # Corrected iteration: iterate over rows using .iterrows()
    for index, entry in results_df[results_df.X_cols_tuple == features_tuple].iterrows():
      plt.plot(dates_test, entry.prediction, label=f":{entry.model_name}", linestyle='--')

    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.legend()
    plt.show()

In [ ]:
def predict_next_days(winning_model, winning_model_name, winning_model_features, future_days=10):

  # Re-create X_train, X_test, y_train, y_test, dates_test with the winning model's features
  # This ensures that X_test has the correct number of features for the winning_model.
  X_train_winning, X_test_winning, y_train_winning, y_test_winning, dates_test_winning = make_datasets(df, winning_model_features, y_col)

  last_historical_block = X_test_winning[-1:] # Select the last sequence from X_test_winning
  last_historical_day = df.iloc[-1].Date
  future_rolling_block = last_historical_block.copy()
  future_predictions = []

  for _ in range(future_days):

    next_pred_day = winning_model.predict(future_rolling_block)[0]
    future_predictions.append(next_pred_day)
    # To simulate new data, shift the window and replace the last element with the new prediction
    # The new prediction (next_pred_day) represents 'y_next', so it becomes the 'y' feature
    # in the next time step's input block. Other features would need to be synthesized or assumed.
    # For simplicity and given the model was trained on ['y', 'NASDAQ'] with 'y_next' as target,
    # we need to ensure the rolling block maintains the correct feature structure.
    # The error suggests a feature mismatch, so for a simple 'y' and 'NASDAQ' model,
    # we need to ensure 'future_rolling_block' is updated correctly.

    # The original implementation `future_rolling_block[:, -1, 0] = next_pred_day`
    # implicitly assumes a single feature is being updated, which is correct for
    # a model trained on only ['y']. For a model trained on ['y', 'NASDAQ'],
    # if 'y' is the first feature (index 0) and 'NASDAQ' is the second (index 1),
    # we can update the 'y' feature and either leave 'NASDAQ' as is or synthesize it.
    # Given the problem's focus on `y_next`, it is reasonable to assume that the 'y' feature
    # in the input sequence is what `next_pred_day` is meant to update.

    # Let's adjust `future_rolling_block` to correctly handle multiple features if the winning model requires it.
    # If the winning model uses multiple features, the prediction `next_pred_day` (which is 'y_next')
    # corresponds to the 'y' feature for the next time step. The other features ('NASDAQ' in this case)
    # would typically need to be forecasted or carried forward.
    # For this simple rolling prediction, we'll assume the predicted 'y' replaces the 'y' feature
    # in the next input window, and other features remain the same from the last historical block
    # or are also predicted/synthesized.

    # For a model with ['y', 'NASDAQ'] features, if the input is (batch, timesteps, features):
    # future_rolling_block should have shape (1, LOOK_BACK_DAYS, num_features).
    # next_pred_day is a single value, representing the predicted 'y_next'.
    # We need to update the 'y' feature in the new time step of the rolling block.
    # Assuming 'y' is the first feature (index 0) and NASDAQ is the second (index 1).
    # The `np.roll` operation shifts the time series data. We then update the last time step's features.

    # Create a new row for the next time step, assuming 'y' is the first feature.
    # If 'winning_model_features' was more complex and included 'NASDAQ',
    # we would need a way to populate the 'NASDAQ' value for the next step.
    # For now, let's assume `next_pred_day` updates the *first* feature (index 0) in the last time step
    # of the rolling window, and if other features exist, they remain as they were in the previous last time step.

    new_input_row = future_rolling_block[0, -1, :].copy() # Copy last row of features
    new_input_row[0] = next_pred_day # Update the 'y' feature (first feature)

    future_rolling_block = np.roll(future_rolling_block, -1, axis=1) # Shift time window
    future_rolling_block[0, -1, :] = new_input_row # Place the new feature vector at the end

  # Calculate min and max for 'y_next' from the original unscaled data (df_orig)
  # This allows us to manually inverse transform the single predicted value.
  # min_y_next_orig = df_orig['y_next_orig'].min()
  # max_y_next_orig = df_orig['y_next_orig'].max()
  # Inverse transform the scaled prediction using the min-max formula
  #np.array(future_predictions) * (max_y_next_orig - min_y_next_orig) + min_y_next_orig
  unscaled_future_predictions = X_scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

  print(f"Using {winning_model_name} and {winning_model_features} to predicti the next {future_days} days:")
  print(f"-------------------------------------------")
  for i, p in enumerate(unscaled_future_predictions, start=1):
    # Convert last_historical_day to datetime object and add days
    prediction_date = pd.to_datetime(last_historical_day) + pd.offsets.BDay(i)
    print(f"{prediction_date.strftime('%Y-%m-%d')}: {float(p):.2f}")

In [ ]:
for stock in project_config['STOCKS']:

  project_config['TKL'] = stock

  results_df, dates_test, unscaled_y_test = race_models(project_config['TKL'], project_config['RACING_MODELS'].split())
  plot_results(results_df, dates_test, unscaled_y_test)

  print(f"AND THE WINNER for {project_config['TKL']} IS ...")
  display(results_df[['model_name', 'X_cols', 'mae', 'r2']].sort_values(by='mae', ascending=True))

  winning_model = results_df.sort_values(by='mae', ascending=True).iloc[0].model
  winning_model_name = results_df.sort_values(by='mae', ascending=True).iloc[0].model_name
  winning_model_features = results_df.sort_values(by='mae', ascending=True).iloc[0].X_cols

  if project_config['chain_notebooks'] == 0
    predict_next_days(winning_model, winning_model_name, winning_model_features, days=10)

  model_path = f"{os.getenv('PROJECT_PATH')}{project_config['pickles_directory']}{project_config['TKL']}.model.{winning_model_name}.{winning_model_features}.keras"
  winning_model.save(model_path)

# Preciding next days (using the Winning Model)